# Exoplanetas

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)   # para visualizar todas las columnas del df
#pd.set_option('display.max_rows', None)   # para visualizar todas las filas del df

import mysql.connector as conn

from selenium import webdriver
import time

### Pasando de pandas a SQL

In [3]:
def crear_db(nombre, user='root', password='password', check=True):
    
    crea_db=conn.connect(host='localhost', user=user, passwd=password)

    cursor=crea_db.cursor()

    cursor.execute(f'drop database if exists {nombre}')
    cursor.execute(f'create database {nombre}')
    
    print('Hecho, base de datos creada. ;)')
    
    if check:
        cursor.execute('show databases')
        for x in cursor:
            print(x)

In [4]:
def crear_tabla_desde_df(df, nombre_tabla, nombre_db, user='root', password='password', check=True, drop=False):
    
    db=conn.connect(host='localhost', user=user, passwd=password, database=nombre_db)
    cursor=db.cursor()
    
    if drop:
        cursor.execute(f'drop table if exists {nombre_tabla}')
    
    tabla=f"create table {nombre_tabla}(id int, {' varchar(200),'.join(df.columns)} varchar(200));"
    print(tabla)
    
    cursor.execute(tabla)
    
    print(f'Hecho, tabla {nombre_tabla} creada. ;)')
    
    if check:
        cursor.execute('show tables')
        for x in cursor:
            print(x)

In [5]:
def insertar_datos(df, nombre_tabla, nombre_db, user='root', password='password', check=True):
    
    db=conn.connect(host='localhost', user=user, passwd=password, database=nombre_db)
    cursor=db.cursor()
    
    for i in range(len(df)):
    
        insert_query='insert into {} (id, {}) values {};'\
                     .format(nombre_tabla, ','.join(df.columns), tuple(np.insert(df.iloc[i].values, 0, i)))

        cursor.execute(insert_query)

    db.commit()
    
    print(f'Hecho, datos dentro de tabla {nombre_tabla}. ;)')
    print()
    print(insert_query)

In [7]:
def leer_sql(query, nombre_tabla, nombre_db, user='root', password='password'):
    
    db=conn.connect(host='localhost', user=user, passwd=password, database=nombre_db)
    
    data=pd.read_sql(query, db)
    
    return data

### 1ª Fuente - Kaggle (formato csv)

https://www.kaggle.com/nasa/kepler-exoplanet-search-results

In [8]:
df=pd.read_csv('../data/cumulative.csv')

df.head()

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_teq_err1,koi_teq_err2,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_tce_delivname,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,-0.002160,0.146,0.318,-0.146,2.95750,0.08190,-0.08190,615.8,19.5,-19.5,2.26,0.26,-0.15,793.0,NaN,NaN,93.59,29.45,-16.65,35.8,1.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,0.586,0.059,-0.443,4.50700,0.11600,-0.11600,874.8,35.5,-35.5,2.83,0.32,-0.19,443.0,NaN,NaN,9.11,2.87,-1.62,25.8,2.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,3,10811496,K00753.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,-0.000581,0.969,5.126,-0.077,1.78220,0.03410,-0.03410,10829.0,171.0,-171.0,14.60,3.92,-1.31,638.0,NaN,NaN,39.30,31.04,-10.49,76.3,1.0,q1_q17_dr25_tce,5853.0,158.0,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,4,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,1.276,0.115,-0.092,2.40641,0.00537,-0.00537,8079.2,12.8,-12.8,33.46,8.50,-2.83,1395.0,NaN,NaN,891.96,668.95,-230.35,505.6,1.0,q1_q17_dr25_tce,5805.0,157.0,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,5,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,-0.001130,0.701,0.235,-0.478,1.65450,0.04200,-0.04200,603.3,16.9,-16.9,2.75,0.88,-0.35,1406.0,NaN,NaN,926.16,874.33,-314.24,40.9,1.0,q1_q17_dr25_tce,6031.0,169.0,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9564 entries, 0 to 9563
Data columns (total 50 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rowid              9564 non-null   int64  
 1   kepid              9564 non-null   int64  
 2   kepoi_name         9564 non-null   object 
 3   kepler_name        2294 non-null   object 
 4   koi_disposition    9564 non-null   object 
 5   koi_pdisposition   9564 non-null   object 
 6   koi_score          8054 non-null   float64
 7   koi_fpflag_nt      9564 non-null   int64  
 8   koi_fpflag_ss      9564 non-null   int64  
 9   koi_fpflag_co      9564 non-null   int64  
 10  koi_fpflag_ec      9564 non-null   int64  
 11  koi_period         9564 non-null   float64
 12  koi_period_err1    9110 non-null   float64
 13  koi_period_err2    9110 non-null   float64
 14  koi_time0bk        9564 non-null   float64
 15  koi_time0bk_err1   9110 non-null   float64
 16  koi_time0bk_err2   9110 

In [10]:
nan=df.isna().sum()

nan[nan>0]

kepler_name          7270
koi_score            1510
koi_period_err1       454
koi_period_err2       454
koi_time0bk_err1      454
koi_time0bk_err2      454
koi_impact            363
koi_impact_err1       454
koi_impact_err2       454
koi_duration_err1     454
koi_duration_err2     454
koi_depth             363
koi_depth_err1        454
koi_depth_err2        454
koi_prad              363
koi_prad_err1         363
koi_prad_err2         363
koi_teq               363
koi_teq_err1         9564
koi_teq_err2         9564
koi_insol             321
koi_insol_err1        321
koi_insol_err2        321
koi_model_snr         363
koi_tce_plnt_num      346
koi_tce_delivname     346
koi_steff             363
koi_steff_err1        468
koi_steff_err2        483
koi_slogg             363
koi_slogg_err1        468
koi_slogg_err2        468
koi_srad              363
koi_srad_err1         468
koi_srad_err2         468
koi_kepmag              1
dtype: int64

In [11]:
df.kepler_name=df.kepler_name.fillna('none')

df.koi_score=df.koi_score.fillna(0)

df=df.drop(columns=['dec', 'koi_teq_err1', 'koi_teq_err2'])

df=df.dropna()

nan=df.isna().sum()

nan[nan>0]

Series([], dtype: int64)

In [12]:
df.columns

Index(['rowid', 'kepid', 'kepoi_name', 'kepler_name', 'koi_disposition',
       'koi_pdisposition', 'koi_score', 'koi_fpflag_nt', 'koi_fpflag_ss',
       'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period', 'koi_period_err1',
       'koi_period_err2', 'koi_time0bk', 'koi_time0bk_err1',
       'koi_time0bk_err2', 'koi_impact', 'koi_impact_err1', 'koi_impact_err2',
       'koi_duration', 'koi_duration_err1', 'koi_duration_err2', 'koi_depth',
       'koi_depth_err1', 'koi_depth_err2', 'koi_prad', 'koi_prad_err1',
       'koi_prad_err2', 'koi_teq', 'koi_insol', 'koi_insol_err1',
       'koi_insol_err2', 'koi_model_snr', 'koi_tce_plnt_num',
       'koi_tce_delivname', 'koi_steff', 'koi_steff_err1', 'koi_steff_err2',
       'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2', 'koi_srad',
       'koi_srad_err1', 'koi_srad_err2', 'ra', 'koi_kepmag'],
      dtype='object')

**A sql...**

In [13]:
crear_db('proceso_etl')

Hecho, base de datos creada. ;)
('app',)
('Apps',)
('information_schema',)
('mydb',)
('mysql',)
('nueva_db',)
('performance_schema',)
('proceso_etl',)
('productos',)
('prueba',)
('pub',)
('publ',)
('publications',)
('sys',)


In [14]:
crear_tabla_desde_df(df, 'kepler', 'proceso_etl')

create table kepler(id int, rowid varchar(200),kepid varchar(200),kepoi_name varchar(200),kepler_name varchar(200),koi_disposition varchar(200),koi_pdisposition varchar(200),koi_score varchar(200),koi_fpflag_nt varchar(200),koi_fpflag_ss varchar(200),koi_fpflag_co varchar(200),koi_fpflag_ec varchar(200),koi_period varchar(200),koi_period_err1 varchar(200),koi_period_err2 varchar(200),koi_time0bk varchar(200),koi_time0bk_err1 varchar(200),koi_time0bk_err2 varchar(200),koi_impact varchar(200),koi_impact_err1 varchar(200),koi_impact_err2 varchar(200),koi_duration varchar(200),koi_duration_err1 varchar(200),koi_duration_err2 varchar(200),koi_depth varchar(200),koi_depth_err1 varchar(200),koi_depth_err2 varchar(200),koi_prad varchar(200),koi_prad_err1 varchar(200),koi_prad_err2 varchar(200),koi_teq varchar(200),koi_insol varchar(200),koi_insol_err1 varchar(200),koi_insol_err2 varchar(200),koi_model_snr varchar(200),koi_tce_plnt_num varchar(200),koi_tce_delivname varchar(200),koi_steff varch

In [15]:
insertar_datos(df, 'kepler', 'proceso_etl')

Hecho, datos dentro de tabla kepler. ;)

insert into kepler (id, rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_tce_delivname,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,koi_kepmag) values (8743, 9564, 10156110, 'K07989.01', 'none', 'FALSE POSITIVE', 'FALSE POSITIVE', 0.0, 0, 0, 1, 1, 4.85603482, 6.356e-05, -6.356e-05, 135.9933, 0.0108, -0.0108, 0.134, 0.323, -0.134, 3.078, 0.283, -0.283, 76.7, 10.8, -10.8, 1.05, 0.36, -0.12, 1266.0, 607.42, 600.39, -194.33, 8.2, 1.0, 'q1_q17_dr25_tce', 6469

In [16]:
leer_sql('select * from kepler limit 3', 'kepler', 'proceso_etl')  # query, tabla, db

,id,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_tce_delivname,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,koi_kepmag
0,0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.0,0,0,0,0,9.48803557,0.00002775,-0.00002775,170.53875,0.00216,-0.00216,0.146,0.318,-0.146,2.9575,0.0819,-0.0819,615.8,19.5,-19.5,2.26,0.26,-0.15,793.0,93.59,29.45,-16.65,35.8,1.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,15.347
1,1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,54.4183827,0.0002479,-0.0002479,162.51384,0.00352,-0.00352,0.586,0.059,-0.443,4.507,0.116,-0.116,874.8,35.5,-35.5,2.83,0.32,-0.19,443.0,9.11,2.87,-1.62,25.8,2.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,15.347
2,2,3,10811496,K00753.01,none,FALSE POSITIVE,FALSE POSITIVE,0.0,0,1,0,0,19.89913995,0.00001494,-0.00001494,175.850252,0.000581,-0.000581,0.969,5.126,-0.077,1.7822,0.0341,-0.0341,10829.0,171.0,-171.0,14.6,3.92,-1.31,638.0,39.3,31.04,-10.49,76.3,1.0,q1_q17_dr25_tce,5853.0,158.0,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,15.436


### 2ª Fuente - Exoplanet API (formato libreria)


http://exoplanet.eu/API/

In [17]:
!pip install pyvo

     |████████████████████████████████| 804 kB 8.9 MB/s eta 0:00:01
  Created wheel for pyvo: filename=pyvo-1.1-py3-none-any.whl size=801573 sha256=33e619c615a6432bd5d6537ce76c2eb1b4e5b9bba77d3beeb2625f2793fa468a
  Stored in directory: /Users/iudh/Library/Caches/pip/wheels/a2/b0/fe/b968e5eca10367d0a4fd30ad640188948e13e1cf57126167c3
  Created wheel for mimeparse: filename=mimeparse-0.1.3-py3-none-any.whl size=3864 sha256=8910116b04a3f7154aab8cbc49665bf318245c925fc6056eff7638f265fd76a7
  Stored in directory: /Users/iudh/Library/Caches/pip/wheels/49/b4/2d/0081759ae1833bd694024801f7aacddcda8a687e8d5fbaeebd
Successfully built pyvo mimeparse


In [18]:
import pyvo

In [19]:
service = pyvo.dal.TAPService("http://voparis-tap-planeto.obspm.fr/tap")

In [20]:
query = "SELECT * FROM exoplanet.epn_core"

In [21]:
results = service.search(query) 

In [22]:
len(results)

4665

In [23]:
results[0]

('11 Com b', 'exoplanet_catalog', '882', 'ci', '11 Com b', 'exoplanet', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 185.1791667, 185.1791667, 17.7927778, 17.7927778, 9.70363596338136e+18, 1.1739363787867e+19, '', nan, nan, nan, nan, nan, nan, 'celestial', nan, nan, nan, nan, nan, nan, '', '', 'phys.size;meta.file#phys.composition.species#phys.mass#phys.mass;stat.error#phys.size.radius#phys.size.radius;stat.error#phys.size.smajAxis#phys.size.smajAxis;stat.error#src.orbital;time.period#src.orbital;time.period;stat.error#src.orbital.eccentricity#src.orbital.eccentricity;stat.error#src.orbital.periastron#src.orbital.periastron;stat.error#time.epoch#time.epoch;src.orbital.periastron#time.epoch;src.orbital.periastron;stat.error#time.epoch;stat.error#src.orbital.inclination#src.orbital.inclination;stat.error#time.publiYear#pos.angDistance#phys.temperature#phys.temperature.effective#phys.gravity#pos.distance#src.spType#phot.mag;em.opt.V#phot.mag;em.opt.I#phot.mag;em.IR.J#phot.m

In [28]:
df=pd.DataFrame(results).dropna(axis=1).dropna()

df=df.drop(columns=['dec', 'measurement_type', 'remarks', 'species'])

for c in df.columns:
    df[c]=df[c].apply(lambda x: str(x).replace('"', '').replace("'", ''))
    
df.head()

,alt_target_name,bib_reference,c1max,c1min,c2max,c2min,creation_date,dataproduct_type,detected_disc,detection_type,discovered,granule_gid,granule_uid,instrument_host_name,instrument_name,magnetic_field,mass_detection_type,modification_date,obs_id,other_web,processing_level,publication_status,publisher,ra,radius_detection_type,release_date,s_region,service_title,spatial_frame_type,star_name,star_spec_type,target_class,target_name,target_region,time_scale,updated
0,,http://adsabs.harvard.edu/abs/2011A%26A...532A...,185.1791667,185.1791667,17.7927778,17.7927778,2012-07-11T13:18:26.124760,ci,,Radial Velocity,2008,exoplanet_catalog,11 Com b,,,False,,2015-08-21T10:53:26.264043,882,,5,Published in a refereed paper,PADC on behalf of LESIA,185.1791667,,2012-07-11T13:18:26.124760,,exoplanet,celestial,11 Com,G8 III,exoplanet,11 Com b,extrasolar planet,UTC,
1,,http://adsabs.harvard.edu/abs/2011A%26A...532A...,245.6041667,245.6041667,-24.0872222,-24.0872222,2012-07-11T13:18:26.124760,ci,,Imaging,2007,exoplanet_catalog,11 Oph b,,,False,,2018-06-18T10:24:14.497285,843,,5,Published in a refereed paper,PADC on behalf of LESIA,245.6041667,,2012-07-11T13:18:26.124760,,exoplanet,celestial,11 Oph,M9,exoplanet,11 Oph b,extrasolar planet,UTC,
2,,http://adsabs.harvard.edu/abs/2011A%26A...532A...,229.275,229.275,71.8238889,71.8238889,2012-07-11T13:18:26.124760,ci,,Radial Velocity,2009,exoplanet_catalog,11 UMi b,,,False,,2009-08-13T00:00:00,575,,5,Published in a refereed paper,PADC on behalf of LESIA,229.275,,2012-07-11T13:18:26.124760,,exoplanet,celestial,11 UMi,K4III,exoplanet,11 UMi b,extrasolar planet,UTC,
3,,http://adsabs.harvard.edu/abs/2011A%26A...532A...,352.8208333,352.8208333,39.2361111,39.2361111,2012-07-11T13:18:26.124760,ci,,Radial Velocity,2008,exoplanet_catalog,14 And b,,,False,,2014-08-27T15:45:24.278413,489,,5,Published in a refereed paper,PADC on behalf of LESIA,352.8208333,,2012-07-11T13:18:26.124760,,exoplanet,celestial,14 And,K0III,exoplanet,14 And b,extrasolar planet,UTC,
4,,http://adsabs.harvard.edu/abs/2011A%26A...532A...,242.5958333,242.5958333,43.8216667,43.8216667,2012-07-11T13:18:26.124760,ci,,Radial Velocity,2002,exoplanet_catalog,14 Her b,,,False,,2014-08-27T15:53:57.679596,77,,5,Published in a refereed paper,PADC on behalf of LESIA,242.5958333,,2012-07-11T13:18:26.124760,,exoplanet,celestial,14 Her,K0 V,exoplanet,14 Her b,extrasolar planet,UTC,


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4665 entries, 0 to 4664
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   alt_target_name        4665 non-null   object
 1   bib_reference          4665 non-null   object
 2   c1max                  4665 non-null   object
 3   c1min                  4665 non-null   object
 4   c2max                  4665 non-null   object
 5   c2min                  4665 non-null   object
 6   creation_date          4665 non-null   object
 7   dataproduct_type       4665 non-null   object
 8   detected_disc          4665 non-null   object
 9   detection_type         4665 non-null   object
 10  discovered             4665 non-null   object
 11  granule_gid            4665 non-null   object
 12  granule_uid            4665 non-null   object
 13  instrument_host_name   4665 non-null   object
 14  instrument_name        4665 non-null   object
 15  magnetic_field       

In [30]:
crear_tabla_desde_df(df, 'exoplanet', 'proceso_etl')

create table exoplanet(id int, alt_target_name varchar(200),bib_reference varchar(200),c1max varchar(200),c1min varchar(200),c2max varchar(200),c2min varchar(200),creation_date varchar(200),dataproduct_type varchar(200),detected_disc varchar(200),detection_type varchar(200),discovered varchar(200),granule_gid varchar(200),granule_uid varchar(200),instrument_host_name varchar(200),instrument_name varchar(200),magnetic_field varchar(200),mass_detection_type varchar(200),modification_date varchar(200),obs_id varchar(200),other_web varchar(200),processing_level varchar(200),publication_status varchar(200),publisher varchar(200),ra varchar(200),radius_detection_type varchar(200),release_date varchar(200),s_region varchar(200),service_title varchar(200),spatial_frame_type varchar(200),star_name varchar(200),star_spec_type varchar(200),target_class varchar(200),target_name varchar(200),target_region varchar(200),time_scale varchar(200),updated varchar(200));
Hecho, tabla exoplanet creada. ;)


In [31]:
insertar_datos(df, 'exoplanet', 'proceso_etl')

Hecho, datos dentro de tabla exoplanet. ;)

insert into exoplanet (id, alt_target_name,bib_reference,c1max,c1min,c2max,c2min,creation_date,dataproduct_type,detected_disc,detection_type,discovered,granule_gid,granule_uid,instrument_host_name,instrument_name,magnetic_field,mass_detection_type,modification_date,obs_id,other_web,processing_level,publication_status,publisher,ra,radius_detection_type,release_date,s_region,service_title,spatial_frame_type,star_name,star_spec_type,target_class,target_name,target_region,time_scale,updated) values (4664, '', 'http://adsabs.harvard.edu/abs/2011A%26A...532A..79S', '308.827231817', '308.827231817', '14.6742132163', '14.6742132163', '2019-01-24T15:17:32.739042', 'ci', '', 'Imaging', '2014', 'exoplanet_catalog', 'zet Del B', '', '', 'False', 'Spectrum', '2019-01-24T15:18:55.966442', '7007', '', '5', 'Published in a refereed paper', 'PADC on behalf of LESIA', '308.827231817', '', '2019-01-24T15:17:32.739042', '', 'exoplanet', 'celestial', 'zet Del A',

In [32]:
leer_sql('select * from exoplanet limit 3', 'exoplanet', 'proceso_etl')

,id,alt_target_name,bib_reference,c1max,c1min,c2max,c2min,creation_date,dataproduct_type,detected_disc,detection_type,discovered,granule_gid,granule_uid,instrument_host_name,instrument_name,magnetic_field,mass_detection_type,modification_date,obs_id,other_web,processing_level,publication_status,publisher,ra,radius_detection_type,release_date,s_region,service_title,spatial_frame_type,star_name,star_spec_type,target_class,target_name,target_region,time_scale,updated
0,0,,http://adsabs.harvard.edu/abs/2011A%26A...532A...,185.1791667,185.1791667,17.7927778,17.7927778,2012-07-11T13:18:26.124760,ci,,Radial Velocity,2008,exoplanet_catalog,11 Com b,,,False,,2015-08-21T10:53:26.264043,882,,5,Published in a refereed paper,PADC on behalf of LESIA,185.1791667,,2012-07-11T13:18:26.124760,,exoplanet,celestial,11 Com,G8 III,exoplanet,11 Com b,extrasolar planet,UTC,
1,1,,http://adsabs.harvard.edu/abs/2011A%26A...532A...,245.6041667,245.6041667,-24.0872222,-24.0872222,2012-07-11T13:18:26.124760,ci,,Imaging,2007,exoplanet_catalog,11 Oph b,,,False,,2018-06-18T10:24:14.497285,843,,5,Published in a refereed paper,PADC on behalf of LESIA,245.6041667,,2012-07-11T13:18:26.124760,,exoplanet,celestial,11 Oph,M9,exoplanet,11 Oph b,extrasolar planet,UTC,
2,2,,http://adsabs.harvard.edu/abs/2011A%26A...532A...,229.275,229.275,71.8238889,71.8238889,2012-07-11T13:18:26.124760,ci,,Radial Velocity,2009,exoplanet_catalog,11 UMi b,,,False,,2009-08-13T00:00:00,575,,5,Published in a refereed paper,PADC on behalf of LESIA,229.275,,2012-07-11T13:18:26.124760,,exoplanet,celestial,11 UMi,K4III,exoplanet,11 UMi b,extrasolar planet,UTC,


### 3ª Fuente - Wikipedia (formato scrapeo)

https://en.wikipedia.org/wiki/List_of_exoplanets_discovered_using_the_Kepler_space_telescope

In [33]:
PATH='driver/chromedriver_mac'

url='https://en.wikipedia.org/wiki/List_of_exoplanets_discovered_using_the_Kepler_space_telescope'

In [38]:
%%time

driver=webdriver.Chrome(PATH)
driver.get(url)

time.sleep(1)

tabla=driver.find_element_by_xpath('//*[@id="mw-content-text"]/div[1]/table[4]')

cabeza=tabla.find_element_by_tag_name('thead')

cabeceras=[e.text\
            .split('\n')[0]\
            .replace('\xad', '')\
            .replace(' ', '_')\
            .replace('.', '')\
            .lower()
           
           for e in cabeza.find_elements_by_tag_name('th')]



tabla=tabla.find_element_by_tag_name('tbody')

filas=tabla.find_elements_by_tag_name('tr')

#elementos=[e.text for e in filas[0].find_elements_by_tag_name('td')] # solo la 1ª fila

elementos=[[e.text for e in f.find_elements_by_tag_name('td')] for f in filas]

elementos

driver.quit()

CPU times: user 27 s, sys: 2.7 s, total: 29.7 s
Wall time: 12min 10s


In [40]:
df=pd.DataFrame(elementos, columns=cabeceras)

df.head()

,planet,discovery_method,mass,radius,density,orbital_period,semimajor_axis,orbital_eccentricity,year_of_confirmation,ref
0,Earth (for reference),,0.00315,0.0892,5.515,365.2563,1,0.0167,—,[19]
1,TrES-2b (Kepler-1b),Transit,1.197,1.247,0.8,2.47061317,0.0367,—,2006,[20]
2,HAT-P-7b (Kepler-2b),Transit,1.781,1.419,0.8,2.2047354,0.0355,0,2008,[21]
3,HAT-P-11b (Kepler-3b),Transit,0.081,0.422,1.33,4.887802443±0.000000034,0.053,0.0198,2009,[22]
4,Kepler-4b,Transit,0.077±0.012,0.357±0.019,1.91+0.36\n−0.47,3.21346 ±0.00022,0.0456±0.0009,0,2010,[23][24]


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2344 entries, 0 to 2343
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   planet                2344 non-null   object
 1   discovery_method      2344 non-null   object
 2   mass                  2344 non-null   object
 3   radius                2344 non-null   object
 4   density               2344 non-null   object
 5   orbital_period        2344 non-null   object
 6   semimajor_axis        2344 non-null   object
 7   orbital_eccentricity  2344 non-null   object
 8   year_of_confirmation  2344 non-null   object
 9   ref                   2344 non-null   object
dtypes: object(10)
memory usage: 183.2+ KB


In [42]:
crear_tabla_desde_df(df, 'wiki', 'proceso_etl')

create table wiki(id int, planet varchar(200),discovery_method varchar(200),mass varchar(200),radius varchar(200),density varchar(200),orbital_period varchar(200),semimajor_axis varchar(200),orbital_eccentricity varchar(200),year_of_confirmation varchar(200),ref varchar(200));
Hecho, tabla wiki creada. ;)
('exoplanet',)
('kepler',)
('wiki',)


In [43]:
insertar_datos(df, 'wiki', 'proceso_etl')

Hecho, datos dentro de tabla wiki. ;)

insert into wiki (id, planet,discovery_method,mass,radius,density,orbital_period,semimajor_axis,orbital_eccentricity,year_of_confirmation,ref) values (2343, 'Kepler-1701b', '', '', '', '', '', '', '', '', '');


In [44]:
leer_sql('select * from wiki limit 3', 'wiki', 'proceso_etl')

,id,planet,discovery_method,mass,radius,density,orbital_period,semimajor_axis,orbital_eccentricity,year_of_confirmation,ref
0,0,Earth (for reference),,0.00315,0.0892,5.515,365.2563,1,0.0167,—,[19]
1,1,TrES-2b (Kepler-1b),Transit,1.197,1.247,0.8,2.47061317,0.0367,—,2006,[20]
2,2,HAT-P-7b (Kepler-2b),Transit,1.781,1.419,0.8,2.2047354,0.0355,0,2008,[21]


### Fusión de los datos

In [45]:
df_kepler=leer_sql('select * from kepler', 'kepler', 'proceso_etl')

df_exo=leer_sql('select * from exoplanet', 'exoplanet', 'proceso_etl')

df_wiki=leer_sql('select * from wiki', 'wiki', 'proceso_etl')

In [47]:
def cambiar_tipo(df):
    
    for c in df.columns:
        try:
            df[c]=df[c].astype(float)
        except:
            continue
            
    return df

In [48]:
df_kepler=cambiar_tipo(df_kepler)

df_exo=cambiar_tipo(df_exo)

df_wiki=cambiar_tipo(df_wiki)

In [59]:
df_wiki.head()

,id,planet,discovery_method,mass,radius,density,orbital_period,semimajor_axis,orbital_eccentricity,year_of_confirmation,ref
0,0.0,Earth (for reference),,0.00315,0.0892,5.515,365.2563,1,0.0167,—,[19]
1,1.0,TrES-2b (Kepler-1b),Transit,1.197,1.247,0.8,2.47061317,0.0367,—,2006,[20]
2,2.0,HAT-P-7b (Kepler-2b),Transit,1.781,1.419,0.8,2.2047354,0.0355,0,2008,[21]
3,3.0,HAT-P-11b (Kepler-3b),Transit,0.081,0.422,1.33,4.887802443±0.000000034,0.053,0.0198,2009,[22]
4,4.0,Kepler-4b,Transit,0.077±0.012,0.357±0.019,1.91+0.36\n−0.47,3.21346 ±0.00022,0.0456±0.0009,0,2010,[23][24]


In [57]:
kepler_grupo=df_kepler.groupby('kepler_name').mean().reset_index().drop('id', axis=1)

kepler_grupo.head()

,kepler_name,rowid,kepid,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,koi_kepmag
0,Kepler-1 b,11.0,11446443.0,0.811,0.0,0.0,0.0,0.0,2.470613,2.700000e-08,-2.700000e-08,122.763305,0.000009,-0.000009,0.818,0.001,-0.001,1.74319,0.00107,-0.00107,14231.0,4.2,-4.2,13.04,0.51,-0.51,1339.0,761.46,106.21,-95.76,4304.3,1.0,5820.0,78.0,-78.0,4.457,0.024,-0.024,0.964,0.038,-0.038,286.80847,11.338
1,Kepler-10 b,3051.0,11904151.0,0.000,1.0,0.0,0.0,0.0,0.837491,3.030000e-07,-3.030000e-07,131.574858,0.000282,-0.000282,0.022,0.388,-0.022,1.79600,0.01040,-0.01040,190.4,1.2,-1.2,1.43,0.08,-0.06,1980.0,3630.32,654.58,-443.61,189.5,2.0,5676.0,85.0,-68.0,4.347,0.030,-0.030,1.044,0.057,-0.042,285.67938,10.961
2,Kepler-10 c,3117.0,11904151.0,0.982,0.0,0.0,0.0,0.0,45.294223,5.600000e-05,-5.600000e-05,138.678725,0.000987,-0.000987,0.025,0.333,-0.025,6.83000,0.03430,-0.03430,477.1,3.7,-3.7,2.26,0.12,-0.09,524.0,17.75,3.21,-2.17,134.8,1.0,5676.0,85.0,-68.0,4.347,0.030,-0.030,1.044,0.057,-0.042,285.67938,10.961
3,Kepler-100 b,2330.0,6521045.0,1.000,0.0,0.0,0.0,0.0,6.887071,2.277000e-05,-2.277000e-05,133.177970,0.002680,-0.002680,0.696,0.029,-0.508,4.46790,0.07570,-0.07570,68.7,2.0,-2.0,1.30,0.08,-0.07,1155.0,420.97,81.61,-63.83,38.2,2.0,5788.0,78.0,-78.0,4.122,0.033,-0.027,1.474,0.094,-0.079,291.38599,11.197
4,Kepler-100 c,2535.0,6521045.0,1.000,0.0,0.0,0.0,0.0,12.815904,1.945000e-05,-1.945000e-05,122.946820,0.001190,-0.001190,0.580,0.005,-0.486,6.38560,0.04380,-0.04380,220.5,2.2,-2.2,2.26,0.15,-0.12,939.0,183.68,35.61,-27.85,106.7,1.0,5788.0,78.0,-78.0,4.122,0.033,-0.027,1.474,0.094,-0.079,291.38599,11.197


In [52]:
df_wiki.shape

(2344, 11)

In [58]:
kepler_grupo.shape

(2274, 43)

**similitud de strings**

https://github.com/seatgeek/fuzzywuzzy

In [60]:
!pip install fuzzywuzzy

In [61]:
from fuzzywuzzy import process, fuzz

In [62]:
fuzz.ratio('Esto es una prueba.', 'Estamos en una prueba.')

88

https://es.wikipedia.org/wiki/Distancia_de_Levenshtein

In [63]:
df_wiki[['match', 'similitud']]=df_wiki.planet.apply(lambda x: process.extractOne(x,
                                                                                 df_kepler.kepler_name.unique(),
                                                                                 scorer=fuzz.token_set_ratio))\
                                                                                .apply(pd.Series)

In [65]:
df_wiki.tail()

,id,planet,discovery_method,mass,radius,density,orbital_period,semimajor_axis,orbital_eccentricity,year_of_confirmation,ref,match,similitud
2339,2339.0,Kepler-1659c,Transit,0.014±0.001,0.17±0.03,—,20.4415±0.0013,0.147280,0.0140,2019,,Kepler-159 c,92
2340,2340.0,Kepler-1660b,Transit timing var.,,,—,,,,2019,,Kepler-660 b,92
2341,2341.0,Kepler-1661b,Transit,0.053±0.038,0.345±0.005,1.6±1.1,175.06±006,0.633±0.005,0.057±0.005,2020,[136],Kepler-661 b,92
2342,2342.0,Kepler-1662b,,,,,,,,,,Kepler-662 b,92
2343,2343.0,Kepler-1701b,,,,,,,,,,Kepler-701 b,92


In [67]:
fusionado=pd.merge(df_wiki, kepler_grupo, left_on='match', right_on='kepler_name')

fusionado.head(15)

,id,planet,discovery_method,mass,radius,density,orbital_period,semimajor_axis,orbital_eccentricity,year_of_confirmation,ref,match,similitud,kepler_name,rowid,kepid,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,koi_kepmag
0,0.0,Earth (for reference),,0.00315,0.0892,5.515,365.2563,1,0.0167,—,[19],Kepler-55 f,40,Kepler-55 f,256.0,8150320.0,0.996,0.0,0.0,0.0,0.0,10.198493,3.250000e-05,-3.250000e-05,137.920670,0.002680,-0.002680,0.714,0.204,-0.473,2.42830,0.07050,-0.07050,634.1,31.3,-31.3,1.75,0.11,-0.06,559.0,23.11,5.35,-3.09,21.4,4.0,4500.0,90.0,-90.0,4.656,0.013,-0.043,0.646,0.043,-0.020,285.16833,15.791
1,125.0,Kepler-55f,Transit,—,0.142±0.009,—,10.198545 ±0.000045,0.081,—,2012,,Kepler-55 f,95,Kepler-55 f,256.0,8150320.0,0.996,0.0,0.0,0.0,0.0,10.198493,3.250000e-05,-3.250000e-05,137.920670,0.002680,-0.002680,0.714,0.204,-0.473,2.42830,0.07050,-0.07050,634.1,31.3,-31.3,1.75,0.11,-0.06,559.0,23.11,5.35,-3.09,21.4,4.0,4500.0,90.0,-90.0,4.656,0.013,-0.043,0.646,0.043,-0.020,285.16833,15.791
2,1.0,TrES-2b (Kepler-1b),Transit,1.197,1.247,0.8,2.47061317,0.0367,—,2006,[20],Kepler-122 e,76,Kepler-122 e,1870.0,4833421.0,1.000,0.0,0.0,0.0,0.0,37.996237,1.515000e-04,-1.515000e-04,162.979840,0.003310,-0.003310,0.023,0.395,-0.023,6.77510,0.09350,-0.09350,403.7,11.0,-11.0,2.39,0.38,-0.38,618.0,34.55,15.66,-12.00,37.8,3.0,6038.0,121.0,-121.0,4.284,0.137,-0.112,1.191,0.189,-0.189,291.11188,14.247
3,280.0,Kepler-122e,Transit,—,0.232±0.048,—,37.993273 ±0.00039,0.227,—,2014,,Kepler-122 e,96,Kepler-122 e,1870.0,4833421.0,1.000,0.0,0.0,0.0,0.0,37.996237,1.515000e-04,-1.515000e-04,162.979840,0.003310,-0.003310,0.023,0.395,-0.023,6.77510,0.09350,-0.09350,403.7,11.0,-11.0,2.39,0.38,-0.38,618.0,34.55,15.66,-12.00,37.8,3.0,6038.0,121.0,-121.0,4.284,0.137,-0.112,1.191,0.189,-0.189,291.11188,14.247
4,2.0,HAT-P-7b (Kepler-2b),Transit,1.781,1.419,0.8,2.2047354,0.0355,0,2008,[21],Kepler-1 b,75,Kepler-1 b,11.0,11446443.0,0.811,0.0,0.0,0.0,0.0,2.470613,2.700000e-08,-2.700000e-08,122.763305,0.000009,-0.000009,0.818,0.001,-0.001,1.74319,0.00107,-0.00107,14231.0,4.2,-4.2,13.04,0.51,-0.51,1339.0,761.46,106.21,-95.76,4304.3,1.0,5820.0,78.0,-78.0,4.457,0.024,-0.024,0.964,0.038,-0.038,286.80847,11.338
5,3.0,HAT-P-11b (Kepler-3b),Transit,0.081,0.422,1.33,4.887802443±0.000000034,0.053,0.0198,2009,[22],Kepler-1 b,75,Kepler-1 b,11.0,11446443.0,0.811,0.0,0.0,0.0,0.0,2.470613,2.700000e-08,-2.700000e-08,122.763305,0.000009,-0.000009,0.818,0.001,-0.001,1.74319,0.00107,-0.00107,14231.0,4.2,-4.2,13.04,0.51,-0.51,1339.0,761.46,106.21,-95.76,4304.3,1.0,5820.0,78.0,-78.0,4.457,0.024,-0.024,0.964,0.038,-0.038,286.80847,11.338
6,146.0,Kepler-64b (PH1 b),Transit,0.531,0.551,—,138.317,0.652,0.07,2013,,Kepler-1 b,89,Kepler-1 b,11.0,11446443.0,0.811,0.0,0.0,0.0,0.0,2.470613,2.700000e-08,-2.700000e-08,122.763305,0.000009,-0.000009,0.818,0.001,-0.001,1.74319,0.00107,-0.00107,14231.0,4.2,-4.2,13.04,0.51,-0.51,1339.0,761.46,106.21,-95.76,4304.3,1.0,5820.0,78.0,-78.0,4.457,0.024,-0.024,0.964,0.038,-0.038,286.80847,11.338
7,193.0,Kepler-86b (PH2 b),Transit,—,0.903,—,282.5255,0.828,0.4,2013,[123],Kepler-1 b,89,Kepler-1 b,11.0,11446443.0,0.811,0.0,0.0,0.0,0.0,2.470613,2.700000e-08,-2.700000e-08,122.763305,0.000009,-0.000009,0.818,0.001,-0.001,1.74319,0.00107,-0.00107,14231.0,4.2,-4.2,13.04,0.51,-0.51,1339.0,761.46,106.21,-95.76,4304.3,1.0,5820.0,78.0,-78.0,4.457,0.024,-0.024,0.964,0.038,-0.038,286.80847,11.338
8,4.0,Kepler-4b,Transit,0.077±0.012,0.357±0.019,1.91+0.36\n−0.47,3.21346 ±0.00022,0.0456±0.0009,0,2010,[23][24],Kepler

In [68]:
fusionado.shape

(2344, 56)

In [69]:
df_wiki.shape

(2344, 13)

In [71]:
crear_tabla_desde_df(fusionado, 'fusion', 'proceso_etl', drop=True)

insertar_datos(fusionado, 'fusion', 'proceso_etl')

leer_sql('select * from fusion limit 3', 'fusion', 'proceso_etl')

create table fusion(id int, id varchar(200),planet varchar(200),discovery_method varchar(200),mass varchar(200),radius varchar(200),density varchar(200),orbital_period varchar(200),semimajor_axis varchar(200),orbital_eccentricity varchar(200),year_of_confirmation varchar(200),ref varchar(200),match varchar(200),similitud varchar(200),kepler_name varchar(200),rowid varchar(200),kepid varchar(200),koi_score varchar(200),koi_fpflag_nt varchar(200),koi_fpflag_ss varchar(200),koi_fpflag_co varchar(200),koi_fpflag_ec varchar(200),koi_period varchar(200),koi_period_err1 varchar(200),koi_period_err2 varchar(200),koi_time0bk varchar(200),koi_time0bk_err1 varchar(200),koi_time0bk_err2 varchar(200),koi_impact varchar(200),koi_impact_err1 varchar(200),koi_impact_err2 varchar(200),koi_duration varchar(200),koi_duration_err1 varchar(200),koi_duration_err2 varchar(200),koi_depth varchar(200),koi_depth_err1 varchar(200),koi_depth_err2 varchar(200),koi_prad varchar(200),koi_prad_err1 varchar(200),koi_p

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'match varchar(200),similitud varchar(200),kepler_name varchar(200),rowid varchar' at line 1

# Este error es por los nombres de las columnas